In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np

In [2]:
class MLPRegressor(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation='relu', output_activation=None, dropout_rate=0.0):
        super(MLPRegressor, self).__init__()

        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Define activation functions
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        else:
            raise ValueError(f"Invalid activation function: {activation}")

        if output_activation == 'sigmoid':
            self.output_activation = nn.Sigmoid()
        elif output_activation == 'tanh':
            self.output_activation = nn.Tanh()
        elif output_activation is None:
            self.output_activation = None
        else:
            raise ValueError(f"Invalid output activation function: {output_activation}")

        # Create the layers
        layers = []
        prev_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(self.activation)
            if dropout_rate > 0.0:
                layers.append(nn.Dropout(dropout_rate))  # Add dropout after activation
            prev_size = hidden_size

        # Output layer
        layers.append(nn.Linear(prev_size, output_size))
        if self.output_activation:
          layers.append(self.output_activation)


        # Combine layers into a sequential model
        self.model = nn.Sequential(*layers)


    def forward(self, x):
        return self.model(x)


def train_model(model, train_loader, criterion, optimizer, num_epochs=10, val_loader=None, patience=None, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train() #set to train mode

    train_loss_history = []
    val_loss_history = []
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, inputs in enumerate(train_loader):
            inputs, labels = inputs["features"].to(device), inputs["target"].to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)
            loss = criterion(outputs.squeeze_(dim=-1), labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_loss_history.append(epoch_loss)

        if verbose:
          print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}", end="")

        # Validation loop (if val_loader is provided)
        if val_loader:
            model.eval()  # Set the model to evaluation mode
            val_loss = 0.0
            with torch.no_grad():
                for inputs in val_loader:
                    inputs, labels = inputs["features"].to(device), inputs["target"].to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs.squeeze_(dim=-1), labels)
                    val_loss += loss.item()
            val_loss /= len(val_loader)
            val_loss_history.append(val_loss)
            if verbose:
                print(f", Val Loss: {val_loss:.4f}", end="")

            # Early stopping check
            if patience is not None:
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    epochs_no_improve = 0
                    # Save the best model so far
                    best_model_state = model.state_dict()
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= patience:
                      if verbose:
                        print(f'\nEarly stopping after {epoch + 1} epochs!')
                      model.load_state_dict(best_model_state)  # Load best model
                      return train_loss_history, val_loss_history, best_val_loss

        if verbose:
          print()
        model.train()  # Set back to train mode

    if val_loader and patience:
      model.load_state_dict(best_model_state)

    return train_loss_history, val_loss_history, best_val_loss if val_loader else None


def predict(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    predictions = []
    target = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs in data_loader:
            inputs = inputs["features"].to(device)
            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            target.append(inputs["target"].cpu().numpy())

    return predictions, target

In [3]:
from utils.preprocessing import TabularDataset, create_dataloader, split_data

In [4]:
batch_size = 512

feature_cols = ['Scalar_B', 'BX_GSE', 'BY_GSM', 'BZ_GSM', 'SW_Temp', 'SW_Density', 'SW_Speed', 'Flow_Pressure', 'E_Field']
target_col = 'Dst'
context_cols = ['time']
numerical_features_info = {k:"standard" for k in feature_cols}

dataset = TabularDataset(
    data_path=r"./dataset/dataset_log.csv",
    feature_cols=feature_cols,
    target_col=target_col,
    context_cols=context_cols,
    numerical_features_info=numerical_features_info
)

train_dataset, val_dataset, test_dataset = split_data(dataset)
train_loader = create_dataloader(train_dataset, batch_size=batch_size)
val_loader = create_dataloader(val_dataset, batch_size=512)
test_loader = create_dataloader(test_dataset, batch_size=512)

In [5]:
# Create the MLP model
input_size = next(iter(train_loader))['features'].shape[1]
hidden_sizes = [64, 32]  # Two hidden layers
output_size = 1  # Regression output
model = MLPRegressor(input_size, hidden_sizes, output_size, activation='relu', dropout_rate=0.2)
# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Train the model
train_loss, val_loss, best_val = train_model(model, train_loader, criterion, optimizer, num_epochs=100, val_loader=val_loader, patience=10)

Epoch 1/100, Loss: 0.9097, Val Loss: 0.9338
Epoch 2/100, Loss: 0.8203, Val Loss: 0.8915
Epoch 3/100, Loss: 0.7536, Val Loss: 0.7019
Epoch 4/100, Loss: 0.6807, Val Loss: 0.6101
Epoch 5/100, Loss: 0.6326, Val Loss: 0.5879
Epoch 6/100, Loss: 0.6027, Val Loss: 0.5478
Epoch 7/100, Loss: 0.6076, Val Loss: 0.5606
Epoch 8/100, Loss: 0.5886, Val Loss: 0.5431
Epoch 9/100, Loss: 0.5997, Val Loss: 0.5417
Epoch 10/100, Loss: 0.5840, Val Loss: 0.5092
Epoch 11/100, Loss: 0.5654, Val Loss: 0.5215
Epoch 12/100, Loss: 0.5624, Val Loss: 0.5647
Epoch 13/100, Loss: 0.5600, Val Loss: 0.5571
Epoch 14/100, Loss: 0.5680, Val Loss: 0.5297
Epoch 15/100, Loss: 0.5598, Val Loss: 0.4974
Epoch 16/100, Loss: 0.5584, Val Loss: 0.6680
Epoch 17/100, Loss: 0.5450, Val Loss: 0.5160
Epoch 18/100, Loss: 0.5396, Val Loss: 0.5551
Epoch 19/100, Loss: 0.5473, Val Loss: 0.5291
Epoch 20/100, Loss: 0.5521, Val Loss: 0.5752
Epoch 21/100, Loss: 0.5481, Val Loss: 0.5060
Epoch 22/100, Loss: 0.5431, Val Loss: 0.5624
Epoch 23/100, Loss:

In [6]:
predictions, target = predict(model, test_loader)


# Evaluate the model (e.g., using R-squared)
from sklearn.metrics import r2_score
r2 = r2_score(target, predictions)
print(f"R-squared on the test set: {r2:.4f}")

# --- Plotting (optional) ---
import matplotlib.pyplot as plt
# Plot training and validation loss
plt.figure(figsize=(8, 6))
plt.plot(train_loss, label='Training Loss')
if val_loader:
    plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Scatter plot of predicted vs. actual values (optional)
plt.figure(figsize=(8, 6))
plt.scatter(target, predictions)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs. Predicted Values')
# Add a diagonal line for perfect predictions
plt.plot([target.min(), target.max()], [target.min(), target.max()], 'k--', lw=2)
plt.show()

IndexError: too many indices for tensor of dimension 2